# Electronics components Detection using YOLOv11

## Introduction

This notebook will guide you through using YOLOv11 (You Only Look Once) Algorithm to detect electronics components such as capacitors, sensors, motor drivers, etc by training the YOLO model on a custom dataset ElectroCom61 which contains over 61 different classes for electronic components.

## Credits

- Dataset: [ElectroCom61](https://data.mendeley.com/datasets/6scy6h8sjz/1)
- Reference: [Roboflow](https://github.com/roboflow/notebooks/blob/main/notebooks/train-yolov8-object-detection-on-custom-dataset.ipynb)
- Model: [Ultralytics](https://github.com/ultralytics/ultralytics)

## Table of Contents

- [Introduction](#introduction)
- [Prerequisites](#prerequisites)
- [Installing YOLOv11](#installing-yolov11)
- [Getting the Dataset](#getting-the-dataset)
- [Sample Inference](#sample-inference)
- [Training](#training)
- [Validation](#validation) 
- [Performance](#performance)  
- [Usage](#usage)

## Prerequisites

We'll be using following tools for this notebook
- Ultralytics YOLOv11 Model
- ElectroCom61 Dataset

Make sure you have access to GPU for faster computation. Run `nvidia-smi` command and check if you get output something like following

In [ ]:
!nvidia-smi

You should get output something like above

Check Current Working Directory Path

In [ ]:
import os
HOME = os.getcwd()
print(HOME)

## Installing YOLOv11

Ultralytics package includes all necessary libraries and dependencies used to run YOLOv11. So installation is quite simple

In [ ]:
!pip install ultralytics

Make sure that ultralytics is installed correctly

In [ ]:
from IPython import display
display.clear_output()

import ultralytics
ultralytics.checks()

## Getting the Dataset

We'll be using ElectroCom61 Dataset for our notebook. The dataset is downloaded from [here](https://data.mendeley.com/datasets/6scy6h8sjz/1)

- Name: ElectroCom61
- Format: YOLOv8
- Images count: 2071
- Image size: 640x640
- Classes count: 61
- Train/Val/Test Distribution: 70%:20%:10%

Download the dataset from given link inside your current working directory and rename the folder to 'Electronics'



## Sample Inference

Now we're all set to run YOLO model. Let's test our installation on a sample image. We'll use a YOLO model pretrained on COCO dataset for inferencing

**NOTE:** There are two ways to use YOLO models
- Using CLI commands
- Using Python code

For more extensibility, we'll be using YOLO model with Python. Read more about CLI method in [Ultralytics YOLO Docs](https://docs.ultralytics.com/usage/cli/)

I'll be using a sample image from Pinterest. You can use any image for this.

**NOTE:** The image used here should contain atleast one entity from COCO classes. Refer [COCO](https://cocodataset.org/) dataset for checking available classes.

In [ ]:
from ultralytics import YOLO
from PIL import Image
import supervision as sv

Let's first see the original image

In [ ]:
sample = Image.open(f'sample.jpg')
sv.plot_image(sample)

Now let's try running pretrained YOLO model on this image

In [ ]:
model = YOLO('yolo11n.pt')
results = model.predict(source=f'sample.jpg', conf=0.25)

Let's plot the detection result what the model found out.

In [ ]:
sv.plot_image(results[0].plot())

Seems like our model has got superpower to see the non-existent things. Let's increase the confidence threshold in parameters to avoid such invalid detections

In [ ]:
new_results = model.predict(source=f'sample.jpg', conf=0.8)

In [ ]:
sv.plot_image(new_results[0].plot())

There you go!❤️

For each detected object, you can find box coordinates, confidence score and class id using following code

In [ ]:
print(f"Box-Coordinates: {new_results[0].boxes.xyxy}")
print(f"Confidence Scores: {new_results[0].boxes.conf}")
print(f"Class IDs: {new_results[0].boxes.cls}")

## Training

Let's now train our model on the custom dataset. Pass the following hyperparameters in the train function
- epochs: Number of epochs
- lr0: Initial learning rate
- lrf: Final learning rate
- imgsz: Square input image size
- batch: Input Batch size
- save: To save the learned weights at checkpoints
- device: To select computation device(CPU/GPU)

In [ ]:
results = model.train(data=f'Electronics/ElectroCom61/data.yaml', epochs=20, lr0=0.01, lrf=0.01, imgsz=640, batch=8, device=0, optimizer='AdamW')

After some trial-and-error, I found the nearly optimal set of hyperparameters.

## Validation

Now that our model is trained, let's check how well does it perform.
mAP is a good indicator to measure model performance

In [ ]:
# Load a model
model = YOLO("runs/detect/train3/weights/best.pt")  # load a custom model

# Validate the model
metrics = model.val()  
metrics.box.map  # map50-95
metrics.box.map50  # map50
metrics.box.map75  # map75
metrics.box.maps  # a list contains map50-95 of each category

## Performance

YOLO provides a detailed visualisation of the model performance. Check out `runs/detect/train` folder for training performance, and `runs/detect/val` folder for validation performance.

We achieved a mAP of 0.899 which is sufficiently reliable for any object detection task.

## Usage

YOLO weights are now upgraded to detect Dataset classes. Now you can use the trained weights for detecting Electronic components.

The trained weights are stored in `runs/detect/train/weights/best.pt` file. To use the trained weights on you images, first load the weights in a YOLO model. Then, use the model to detect objects in your images.

In [ ]:
model = YOLO('runs/detect/train3/weights/best.pt')

result = model.predict(source='Electronics/ElectroCom61/test/images/IMG_5229_JPG.rf.716e8ad846e9bc2c9a2fc21266bf280e.jpg')

I have used an image from test images for testing the trained weights. Lets check how well our model performs.

In [ ]:
sv.plot_image(result[0].plot())

## Portable Weights

Now you can use you trained model any any machine with ultralytics installed. Just copy the weights `runs/detect/train/weights/best.pt` to your machine and load the YOLO model with those weights.

Your model is now ready to _look_ at the images